<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/omake05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML omake05

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png">


----
# K-平均法の画像データ圧縮への応用
----

以前，画像の画素値を $K$-平均法でクラスタリングして減色する実験を行いました．
このような減色を行うと，画像のデータ量を削減することができます．
一般的な画像の画素値は，RGBそれぞれの値が $0$ から $255$ の整数で表されるので，$8 \times 3 = 24$ bitのデータ量があります．これを仮に $64$ 色に減色したとすると，$64 = 2^6$ より一つの画素を $6$ bit で表せます．つまり，$64$色に減色すると画像全体のデータ量を $1/4$ にできます．

このおまけ課題では，一つの画素の単位ではなく画像の小さな領域をひとまとめに扱ってクラスタリングしてみます．
例えば，画像を縦横8画素の小さいブロックに分割して，その単位でクラスタリングする，というようなことです．一つの画素の単位の場合，クラスタリングの対象となるデータは$3$次元でしたが，8x8画素のブロックの場合は$8\times 8 \times 3 = 192$ 次元となります．
どんな結果が得られるか，やってみましょう．

---
## 準備

画像をブロックに分割したりする処理は，こちらで用意した関数使って楽をしてもらうことにします．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# コンピュータビジョンライブラリ OpenCV
import cv2

In [ ]:
# Colab へのファイルアップロードを実行する関数
#
def uploadToColab():
    try:
        from google.colab import files
        rv = files.upload()
    except:
        print('このコードは Colab 以外の環境では実行できないよ．')


In [ ]:
# OpenCVの形式の画像を表示する関数
#
def imshow(img):
    try:
        from google.colab.patches import cv2_imshow
        cv2_imshow(img) # Colab上で実行している場合
    except:
        cv2.imshow(img)  # それ以外の場合

In [ ]:
### 画像を bsize x bsize のブロックに分割する関数
#
def image2blocks(img, bsize):

    assert img.ndim == 3 and img.shape[2] == 3, 'img の shape がおかしい？？'

    # 画像サイズが bsize の整数倍になるよう端を切り捨てる
    h, w = img.shape[:2]
    ny, nx = h//bsize, w//bsize
    h2, w2 = ny*bsize, nx*bsize
    img2 = img[:h2, :w2]

    # ブロック切り出し
    blocks = np.empty((ny, nx, bsize*bsize*3), dtype=img2.dtype)
    for iy in range(ny):
        for ix in range(nx):
            blocks[iy, ix, :] = img2[iy*bsize:(iy+1)*bsize, ix*bsize:(ix+1)*bsize, :].reshape(-1)

    return blocks


In [ ]:
### ブロックから画像を復元する関数
#
def blocks2image(blocks, bsize):

    assert blocks.ndim == 3 and blocks.shape[2] == bsize*bsize*3, 'blocks の shape がおかしい？'

    ny, nx = blocks.shape[:2]
    img = np.empty((ny*bsize, nx*bsize, 3), dtype=blocks.dtype)

    # ブロックから画像へ
    for iy in range(ny):
        for ix in range(nx):
            img[iy*bsize:(iy+1)*bsize, ix*bsize:(ix+1)*bsize, :] = blocks[iy, ix, :].reshape((bsize, bsize, -1))

    return img


In [ ]:
#####  セントロイドを可視化する関数
#
def displayCentroid(ax, centroid, nx, ny, bsize, gap=1):

    assert centroid.ndim == 2 and centroid.shape[1] == bsize*bsize*3

    # 並べた画像の幅と高さ
    width  = nx * (bsize + gap) + gap
    height = ny * (bsize + gap) + gap

    # 画像の作成
    img = np.zeros((height, width, 3), dtype=np.uint8) + 128
    for iy in range(ny):
        lty = iy*(bsize + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= centroid.shape[0]:
                break
            ltx = ix*(bsize + gap) + gap
            img[lty:lty+bsize, ltx:ltx+bsize, :] = centroid[iy*nx+ix, :].reshape((bsize, bsize, 3))

    # centroid は OpenCV 形式（画素値がBGRの順）を想定してるが，matplotlib は RGB の順なので変換
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 表示
    ax.axis('off')
    ax.imshow(img2)

---
## 実験0

上記の関数の使い方を理解するために，簡単な実験を．

In [ ]:
# お試し用画像
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/uni3.png

In [ ]:
# 画像を読み込んで表示
img = cv2.imread('uni3.png')
assert img.ndim == 3 and img.shape[2] == 3 # 3チャンネルカラー画像のみ扱えます
imshow(img)

In [ ]:
print('img.shape:', img.shape)

# bsize x bsize にブロック分割
bsize = 8
X = image2blocks(img, bsize)
print('X.shape:', X.shape)

# データ数 x 次元数 の2次元配列に reshape
XX = X.reshape((-1, bsize*bsize*3))
print('XX.shape:', XX.shape)

# XX と同じ形，型の配列つくる
YY = np.empty_like(XX)
print('YY.shape:', XX.shape)

# XX の行ごとに列方向の平均値をもとめてそれを YY の同じ場所に代入
for n in range(XX.shape[0]):
    YY[n, :] = int(np.mean(XX[n, :]) + 0.5) # 平均値は浮動小数点数なので四捨五入して代入

# YY を X と同じ shape にする
Y = YY.reshape(X.shape)
print('Y.shape:', Y.shape)

# Y を画像の形に
imgY = blocks2image(Y, bsize)
print('imgY.shape:', imgY.shape)

# 画像として表示
imshow(imgY)

上のコードセルを実行すると，$8\times 8$ のブロックごとにその中の画素値の平均を求め，それをブロック内のすべての位置に代入した画像が得られます．RGB全てに同じ値が代入されるので，グレイスケール画像に見えるはずです．

`img`, `X`, `XX` 等の shape がどうなっているか確かめて，何をやっているのか理解しましょう．

---
## 実験1

上記の関数たちを利用して，自分で用意した画像を $8\times 8$ ブロックで $K$-means クラスタリングし，各ブロックを対応するセントロイドに置き換えた画像を作る実験をしましょう．



In [ ]:
# Colab へファイルをアップロード
uploadToColab()

# ls コマンドでファイルを一覧
! ls

In [ ]:
# 画像を読み込む．`uni3.png` を自分がアップロードしたファイルの名前に修正
img = cv2.imread('uni3.png')
assert img.ndim == 3 and img.shape[2] == 3 # 3チャンネルカラー画像のみ扱えます
imshow(img)
print('img.shape:', img.shape)

以下にコードを書きましょう．

実験0のコードの `# XX の行ごとに列方向の平均値をもとめてそれを YY の同じ場所に代入` の部分を書き換えて，次のような処理をさせることになります．

1. `XX` を学習データとして K-means クラスタリングの学習を行う
1. セントロイドを得る
1. `XX` の行ごとにラベル（セントロイドの番号）を得る
1. `YY` の各行に対応するセントロイドの値を代入する

K-means クラスタリングは，機械学習ライブラリ [scikit-learn](https://scikit-learn.org/) の [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) を使ってやるのがよいでしょう．
画像の減色のコードも参考にしたらよいかもしれません．

セントロイドの値が `centroid` という名前で `(クラスタ数, 次元数)` という shape をもつ2次元配列に格納されているときに次のコードセルを実行すると，セントロイド（のうち先頭の100個）の値を可視化することができます．

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
displayCentroid(ax, centroid, 10, 10, bsize)
plt.show()

`bsize = 8` で `K = 256` の場合，画像データは何分の一に圧縮することになるでしょうか？